# Swing Vision Transformation
#### Converting SwingVision data into UCLA Tennis Consulting format

## Notebook Start

In [1]:
import pandas as pd
import numpy as np
import os 
import re

# Option to display max rows/columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Load in data

In [2]:
# Input file name here
your_file_name = 'Rudy_Youcef_FSU_sv.xlsx'
swing_data = pd.read_excel(your_file_name, sheet_name='Shots')
swing_data.shape

(677, 25)

In [3]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
0,Rudy Quan,1,first_serve,Serve,Flat,78.818588,1,1,1,short,deuce,near,0.375670,6.704057,out,deuce,far,-1.587323,22.818127,2.682724,down the T,In,False,21:13:17,12.080000
1,Youcef Rihane,2,first_return,Backhand,Topspin,51.792099,1,1,1,deep,deuce,far,-0.948239,19.035049,out,deuce,near,1.339200,-1.593285,0.891651,inside out,In,False,21:13:18,13.040000
2,Rudy Quan,3,serve_plus_one,Forehand,Topspin,58.796837,1,1,1,deep,ad,near,-1.727404,0.906123,deep,deuce,far,-1.565159,24.191404,1.197903,down the line,In,False,21:13:19,14.300000
3,Youcef Rihane,4,return_plus_one,Backhand,Topspin,49.338074,1,1,1,out,ad,far,2.808425,24.438177,out,ad,near,-1.930130,-1.595187,0.957120,cross court,Out,False,21:13:20,15.500000
4,Rudy Quan,5,in_play,Backhand,Flat,56.885868,1,1,1,deep,ad,near,-2.039777,4.713785,deep,ad,far,2.808425,24.145630,0.636929,cross court,In,False,21:13:21,16.959999


### Clean swing vision data

In [4]:
swing_data.query('Stroke == "Feed"').head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
30,Youcef Rihane,0,none,Feed,Flat,19.052607,5,1,1,short,deuce,far,-0.485830,18.041225,short,deuce,near,1.002303,7.083730,1.791148,---,In,False,21:15:11,126.760002
61,Rudy Quan,0,none,Feed,Flat,20.784662,10,2,1,short,deuce,far,-1.249201,16.799627,deep,deuce,near,1.514472,0.542222,0.617660,---,In,False,21:18:24,319.549988
74,Youcef Rihane,0,none,Feed,Flat,23.382744,12,3,1,short,ad,near,-0.473872,6.239765,deep,deuce,far,-3.208965,20.970543,2.106119,---,In,False,21:19:30,385.119995
75,Youcef Rihane,1,none,Feed,Flat,18.186579,12,3,1,deep,ad,near,-2.165251,3.997676,deep,deuce,far,-3.153087,21.960804,2.471642,---,In,False,21:19:38,393.329987
80,Youcef Rihane,0,none,Feed,Flat,20.784662,13,3,1,deep,ad,near,-2.438261,4.502622,short,deuce,far,-3.327099,16.743202,1.614205,---,In,False,21:20:19,434.989990


In [5]:
swing_data.query('Shot == 0').head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
30,Youcef Rihane,0,none,Feed,Flat,19.052607,5,1,1,short,deuce,far,-0.485830,18.041225,short,deuce,near,1.002303,7.083730,1.791148,---,In,False,21:15:11,126.760002
61,Rudy Quan,0,none,Feed,Flat,20.784662,10,2,1,short,deuce,far,-1.249201,16.799627,deep,deuce,near,1.514472,0.542222,0.617660,---,In,False,21:18:24,319.549988
74,Youcef Rihane,0,none,Feed,Flat,23.382744,12,3,1,short,ad,near,-0.473872,6.239765,deep,deuce,far,-3.208965,20.970543,2.106119,---,In,False,21:19:30,385.119995
80,Youcef Rihane,0,none,Feed,Flat,20.784662,13,3,1,deep,ad,near,-2.438261,4.502622,short,deuce,far,-3.327099,16.743202,1.614205,---,In,False,21:20:19,434.989990
87,Youcef Rihane,0,none,Feed,Flat,22.516716,14,3,1,short,ad,near,-1.952927,7.803154,deep,deuce,far,-2.221235,21.249830,2.106010,---,In,False,21:20:48,463.369995


In [6]:
swing_data.query('Type == "none"').head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
30,Youcef Rihane,0,none,Feed,Flat,19.052607,5,1,1,short,deuce,far,-0.485830,18.041225,short,deuce,near,1.002303,7.083730,1.791148,---,In,False,21:15:11,126.760002
61,Rudy Quan,0,none,Feed,Flat,20.784662,10,2,1,short,deuce,far,-1.249201,16.799627,deep,deuce,near,1.514472,0.542222,0.617660,---,In,False,21:18:24,319.549988
74,Youcef Rihane,0,none,Feed,Flat,23.382744,12,3,1,short,ad,near,-0.473872,6.239765,deep,deuce,far,-3.208965,20.970543,2.106119,---,In,False,21:19:30,385.119995
75,Youcef Rihane,1,none,Feed,Flat,18.186579,12,3,1,deep,ad,near,-2.165251,3.997676,deep,deuce,far,-3.153087,21.960804,2.471642,---,In,False,21:19:38,393.329987
77,Youcef Rihane,3,none,Backhand,Flat,42.652466,12,3,1,short,deuce,near,2.457301,9.337544,out,deuce,far,-2.024198,26.844564,1.261113,inside out,In,False,21:20:00,415.619995


#### Remove all the rows where it is a feed

In [7]:
rows_to_drop = swing_data.query('Type == "none"').index
swing_data = swing_data.drop(rows_to_drop)
swing_data = swing_data.reset_index(drop=True) # Important to reindex to avoid missing indicies
swing_data.shape

(615, 25)

In [8]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time
0,Rudy Quan,1,first_serve,Serve,Flat,78.818588,1,1,1,short,deuce,near,0.375670,6.704057,out,deuce,far,-1.587323,22.818127,2.682724,down the T,In,False,21:13:17,12.080000
1,Youcef Rihane,2,first_return,Backhand,Topspin,51.792099,1,1,1,deep,deuce,far,-0.948239,19.035049,out,deuce,near,1.339200,-1.593285,0.891651,inside out,In,False,21:13:18,13.040000
2,Rudy Quan,3,serve_plus_one,Forehand,Topspin,58.796837,1,1,1,deep,ad,near,-1.727404,0.906123,deep,deuce,far,-1.565159,24.191404,1.197903,down the line,In,False,21:13:19,14.300000
3,Youcef Rihane,4,return_plus_one,Backhand,Topspin,49.338074,1,1,1,out,ad,far,2.808425,24.438177,out,ad,near,-1.930130,-1.595187,0.957120,cross court,Out,False,21:13:20,15.500000
4,Rudy Quan,5,in_play,Backhand,Flat,56.885868,1,1,1,deep,ad,near,-2.039777,4.713785,deep,ad,far,2.808425,24.145630,0.636929,cross court,In,False,21:13:21,16.959999


### Load in Points data

In [9]:
swing_data_points = pd.read_excel(your_file_name, sheet_name='Points')
swing_data_points.shape

(97, 15)

In [10]:
def create_point(server, player1score, player2score):
    if server == "host":
        return str(player1score) + "-" + str(player2score)
    else:
        return str(player2score) + "-" + str(player1score)
    
swing_data_points['pointScore'] = swing_data_points.apply(lambda x: create_point(x['Match Server'], x['Host Game Score'], x['Guest Game Score']), axis=1)

In [11]:
swing_data_points = swing_data_points.rename(columns={'Break Point' : 'isBreakPoint'})
swing_data_points['isBreakPoint'] = swing_data_points['isBreakPoint'].replace(False, '')
swing_data_points['isBreakPoint'] = swing_data_points['isBreakPoint'].replace(True, 1)

In [12]:
swing_data_points = swing_data_points[['Point', 'pointScore', 'isBreakPoint']]
swing_data_points.head()

,Point,pointScore,isBreakPoint
0,1,0-0,
1,2,0-15,
2,3,0-30,
3,4,0-40,1
4,5,15-40,1


In [13]:
swing_data = pd.merge(swing_data, swing_data_points, on='Point')

### Load in Games data

In [14]:
swing_data_games = pd.read_excel(your_file_name, sheet_name='Games')
swing_data_games.shape

(18, 9)

In [15]:
swing_data_games.head()

,Game,Set,Server,Host Set Score,Guest Set Score,Game Winner,Start Time,Video Time,Duration
0,1,1,host,0,0,guest,21:13:19,14.080000,131.729996
1,2,1,guest,0,1,host,21:15:30,145.800003,233.419998
2,3,1,host,1,1,host,21:19:24,379.220001,139.190002
3,4,1,guest,2,1,guest,21:21:43,518.409973,190.589996
4,5,1,host,2,2,host,21:24:54,709.000000,199.179993


In [16]:
def create_game(player1game, player2game):
        return str(player1game) + "-" + str(player2game)

    
swing_data_games['gameScore'] = swing_data_games.apply(lambda x: create_game(x['Host Set Score'], x['Guest Set Score']),  axis=1)

In [17]:
swing_data_games

,Game,Set,Server,Host Set Score,Guest Set Score,Game Winner,Start Time,Video Time,Duration,gameScore
0,1,1,host,0,0,guest,21:13:19,14.080000,131.729996,0-0
1,2,1,guest,0,1,host,21:15:30,145.800003,233.419998,0-1
2,3,1,host,1,1,host,21:19:24,379.220001,139.190002,1-1
3,4,1,guest,2,1,guest,21:21:43,518.409973,190.589996,2-1
4,5,1,host,2,2,host,21:24:54,709.000000,199.179993,2-2
5,6,1,guest,3,2,guest,21:28:13,908.179993,223.210007,3-2
6,7,1,host,3,3,host,21:31:56,1131.390015,218.320007,3-3
7,8,1,guest,4,3,guest,21:35:34,1349.699951,221.830002,4-3
8,9,1,host,4,4,host,21:39:16,1571.530029,165.860001,4-4
9,10,1,guest,5,4,host,21:42:02,1737.400024,283.549988,5-4


In [18]:
swing_data_games = swing_data_games[['Game', 'gameScore']]

In [19]:
swing_data = pd.merge(swing_data, swing_data_games, on="Game")

In [20]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,pointScore,isBreakPoint,gameScore
0,Rudy Quan,1,first_serve,Serve,Flat,78.818588,1,1,1,short,deuce,near,0.375670,6.704057,out,deuce,far,-1.587323,22.818127,2.682724,down the T,In,False,21:13:17,12.080000,0-0,,0-0
1,Youcef Rihane,2,first_return,Backhand,Topspin,51.792099,1,1,1,deep,deuce,far,-0.948239,19.035049,out,deuce,near,1.339200,-1.593285,0.891651,inside out,In,False,21:13:18,13.040000,0-0,,0-0
2,Rudy Quan,3,serve_plus_one,Forehand,Topspin,58.796837,1,1,1,deep,ad,near,-1.727404,0.906123,deep,deuce,far,-1.565159,24.191404,1.197903,down the line,In,False,21:13:19,14.300000,0-0,,0-0
3,Youcef Rihane,4,return_plus_one,Backhand,Topspin,49.338074,1,1,1,out,ad,far,2.808425,24.438177,out,ad,near,-1.930130,-1.595187,0.957120,cross court,Out,False,21:13:20,15.500000,0-0,,0-0
4,Rudy Quan,5,in_play,Backhand,Flat,56.885868,1,1,1,deep,ad,near,-2.039777,4.713785,deep,ad,far,2.808425,24.145630,0.636929,cross court,In,False,21:13:21,16.959999,0-0,,0-0


### Load in Sets data

In [21]:
swing_data_sets = pd.read_excel(your_file_name, sheet_name='Sets')
swing_data_sets.shape

(2, 10)

In [22]:
swing_data_sets

,Set,Host Score,Guest Score,Host Tiebreak Score,Guest Tiebreak Score,Set Winner,Super Tiebreak,Start Time,Video Time,Duration
0,1,6,4,0,0,host,False,21:13:19,14.080000,2006.869995
1,2,6,2,0,0,host,False,21:46:45,2020.949951,1990.530029


In [23]:
host_set_score = 0
guest_set_score = 0

def create_set(set_winner):
        global host_set_score, guest_set_score  # Declare global variables
        if set_winner == "host":
                host_set_score += 1
        else:        
                guest_set_score += 1
        
        return str(host_set_score) + "-" + str(guest_set_score)

swing_data_sets['setScore'] = None
swing_data_sets.at[0, 'setScore'] = "0-0"
    
swing_data_sets.iloc[1:, swing_data_sets.columns.get_loc('setScore')] = swing_data_sets.iloc[1:].apply(lambda x: create_set(x['Set Winner']),  axis=1)


In [24]:
swing_data_sets = swing_data_sets[['Set', 'setScore']]


In [25]:
swing_data = pd.merge(swing_data, swing_data_sets, on="Set")
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,pointScore,isBreakPoint,gameScore,setScore
0,Rudy Quan,1,first_serve,Serve,Flat,78.818588,1,1,1,short,deuce,near,0.375670,6.704057,out,deuce,far,-1.587323,22.818127,2.682724,down the T,In,False,21:13:17,12.080000,0-0,,0-0,0-0
1,Youcef Rihane,2,first_return,Backhand,Topspin,51.792099,1,1,1,deep,deuce,far,-0.948239,19.035049,out,deuce,near,1.339200,-1.593285,0.891651,inside out,In,False,21:13:18,13.040000,0-0,,0-0,0-0
2,Rudy Quan,3,serve_plus_one,Forehand,Topspin,58.796837,1,1,1,deep,ad,near,-1.727404,0.906123,deep,deuce,far,-1.565159,24.191404,1.197903,down the line,In,False,21:13:19,14.300000,0-0,,0-0,0-0
3,Youcef Rihane,4,return_plus_one,Backhand,Topspin,49.338074,1,1,1,out,ad,far,2.808425,24.438177,out,ad,near,-1.930130,-1.595187,0.957120,cross court,Out,False,21:13:20,15.500000,0-0,,0-0,0-0
4,Rudy Quan,5,in_play,Backhand,Flat,56.885868,1,1,1,deep,ad,near,-2.039777,4.713785,deep,ad,far,2.808425,24.145630,0.636929,cross court,In,False,21:13:21,16.959999,0-0,,0-0,0-0


### Create shot data csv

In [26]:
# Check existing columns
swing_data.columns

Index(['Player', 'Shot', 'Type', 'Stroke', 'Spin', 'Speed (MPH)', 'Point',
       'Game', 'Set', 'Bounce Depth', 'Bounce Zone', 'Bounce Side',
       'Bounce (x)', 'Bounce (y)', 'Hit Depth', 'Hit Zone', 'Hit Side',
       'Hit (x)', 'Hit (y)', 'Hit (z)', 'Direction', 'Result', 'Favorited',
       'Start Time', 'Video Time', 'pointScore', 'isBreakPoint', 'gameScore',
       'setScore'],
      dtype='object')

In [27]:
swing_data.head()

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,Bounce Zone,Bounce Side,Bounce (x),Bounce (y),Hit Depth,Hit Zone,Hit Side,Hit (x),Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,pointScore,isBreakPoint,gameScore,setScore
0,Rudy Quan,1,first_serve,Serve,Flat,78.818588,1,1,1,short,deuce,near,0.375670,6.704057,out,deuce,far,-1.587323,22.818127,2.682724,down the T,In,False,21:13:17,12.080000,0-0,,0-0,0-0
1,Youcef Rihane,2,first_return,Backhand,Topspin,51.792099,1,1,1,deep,deuce,far,-0.948239,19.035049,out,deuce,near,1.339200,-1.593285,0.891651,inside out,In,False,21:13:18,13.040000,0-0,,0-0,0-0
2,Rudy Quan,3,serve_plus_one,Forehand,Topspin,58.796837,1,1,1,deep,ad,near,-1.727404,0.906123,deep,deuce,far,-1.565159,24.191404,1.197903,down the line,In,False,21:13:19,14.300000,0-0,,0-0,0-0
3,Youcef Rihane,4,return_plus_one,Backhand,Topspin,49.338074,1,1,1,out,ad,far,2.808425,24.438177,out,ad,near,-1.930130,-1.595187,0.957120,cross court,Out,False,21:13:20,15.500000,0-0,,0-0,0-0
4,Rudy Quan,5,in_play,Backhand,Flat,56.885868,1,1,1,deep,ad,near,-2.039777,4.713785,deep,ad,far,2.808425,24.145630,0.636929,cross court,In,False,21:13:21,16.959999,0-0,,0-0,0-0


In [28]:
# add in all desired column labels, with swingvision labels at end

columm_names = (['pointScore', 'gameScore', 'setScore', 'isPointStart', 
                                         'pointStartTime', 'isPointEnd', 'pointEndTime','pointNumber',
                                         'isBreakPoint','shotInRally','side','serverName',
                                         'serverFarNear','firstServeIn','firstServeZone',
                                         'firstServeXCoord','firstServeYCoord',
                                         'secondServeIn','secondServeZone','secondServeXCoord',
                                         'secondServeYCoord','isAce','shotContactX',
                                         'shotContactY','shotDirection','shotFhBh',
                                         'isSlice','isVolley','isOverhead','isApproach','isDropshot',
                                         'isExcitingPoint','atNetPlayer1','atNetPlayer2','isLob',
                                         'shotLocationX','shotLocationY','isWinner','isErrorWideR', 'isErrorWideL',
                                         'isErrorNet','isErrorLong','clientTeam',
                                         'Date', 'Division', 'Event', 'lineupPosition','matchDetails',
                                         'matchVenue' , 'opponentTeam', 
                                         'player1Name', 'player2Name','player1Hand','player2Hand',
                                        'Round','Surface','Notes'])

shot_data = pd.DataFrame(columns=columm_names)
shot_data

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes


### Score Columns

In [29]:
shot_data['pointScore'] = swing_data['pointScore']
shot_data['gameScore'] = swing_data['gameScore']
shot_data['setScore'] = swing_data['setScore']

In [30]:
shot_data.head(10)

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes
0,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0-0,0-0,0-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### isPointStart and isPointEnd columns 

In [31]:
def assign_pointstart(x):
    if (x == 'first_serve') | (x == 'second_serve'):
        return 1
    
    return ''

shot_data['isPointStart'] = swing_data['Type'].apply(assign_pointstart)


index_list = []

for i in swing_data['Point'].unique().tolist():
    last_point_index = swing_data[swing_data['Point'] == i].index[-1]
    index_list.append(last_point_index)
    
shot_data.loc[index_list,'isPointEnd'] = 1
shot_data['isPointEnd'] = shot_data['isPointEnd'].fillna('')

### pointStartTime and pointEndTime Columns

In [32]:
# Grab the initial time
match_start_time = re.split(":", swing_data["Start Time"][0])

for i in range(len(swing_data)): # converting into milliseconds
    time = re.split(":", swing_data["Start Time"][i])
    shot_data.loc[i,'pointStartTime'] = ((int(time[0]) - int(match_start_time[0])) *  3600000
                                       + (int(time[1]) - int(match_start_time[1])) *  60000
                                       + (int(time[2]) - int(match_start_time[2])) *  1000)
    
# Assigns last shot time to pointEndTime column
shot_data['pointEndTime'] = np.where(shot_data['isPointEnd'] == 1, shot_data['pointStartTime'], '')

### pointNumber Column

In [33]:
shot_data['pointNumber'] = swing_data['Point']

### isBreakPoint Column

In [34]:
shot_data['isBreakPoint'] = swing_data['isBreakPoint']

### shotInRally column

In [35]:
shot_data.shotInRally = swing_data.Shot

### side Column

In [36]:
def side(x, side, xcoord):
    if 'deuce' in x:
        return 'Deuce'
    elif 'ad' in x:
        return 'Ad'
    elif 'center_line' in x: # unique values include deuce, ad and center_line
        if (side == 'near') & (xcoord > 0):
            return 'Deuce'
        else:
            return 'Ad'
    else:
        return ''

shot_data['side'] = swing_data.apply(lambda x: side(x['Hit Zone'], x['Hit Side'], x['Bounce (x)']), axis = 1)

### Players

In [37]:
# ucla roster 23-24 men and womens
ucla_roster_23 = ["Gianluca Ballotta", 
                   "Jeffrey Fradkin", 
                   "Alexander Hoogmartens",
                   "Spencer Johnson",
                   "Stefan Leustian",
                   "Timothy Li",
                   "Govind Nanda",
                   "Jorge Plans Gonzalez",
                   "Giacomo Revelli",
                   "Aadarsh Tripathi",
                   "Emon van Loben Sels",
                   "Azuma Visaya",
                   "Rudy Quan",
                   "Leo Von Bismark",
                   
                   "Tian Fangran",
                   "Bianca Fernandez",
                   "Ahmani Guichard",
                   "Kimmi Hance",
                   "Mia Jovic",
                   "Anne-Christine Lutkemeyer",
                   "Vanessa Ong",
                   "Sasha Vagramov",
                   "Elise Wagle"]

In [38]:
# list of names who are playing in match
players = swing_data['Player'].unique()

# checks which one is UCLA player
is_ucla_player = [any([name in roster_name for roster_name in ucla_roster_23]) for name in players]

In [39]:
# assigns ucla player to player 1, and non ucla to player 2
shot_data.loc[0, "player1Name"] = players[is_ucla_player]
shot_data.loc[0, "player2Name"] = players[np.invert(is_ucla_player)]

### serverName Column

In [40]:
def assign_server_name(stroke, server):
    if stroke != 'Serve':
        return ''
    
    if server.startswith(players[is_ucla_player][0]):
        return 'Player1'
    elif server.startswith(players[np.invert(is_ucla_player)][0]):
        return 'Player2'
    
shot_data['serverName'] = swing_data.apply(lambda x: assign_server_name(x['Stroke'], x['Player']), axis=1)
shot_data['serverName'].replace(['', 'na'], pd.NaT, inplace=True)
shot_data['serverName'] = shot_data['serverName'].ffill()

### serverFarNear Column

In [41]:
shot_data.serverFarNear = np.where((swing_data.Stroke == 'Serve'), np.where(swing_data['Hit Side'] == 'far', 'Far', 'Near'), '')
shot_data['serverFarNear'].replace(['', 'na'], pd.NaT, inplace=True)
shot_data['serverFarNear'] = shot_data['serverFarNear'].ffill()

### firstServeIn and secondServeIn Columns

In [42]:
shot_data.firstServeIn = np.where((swing_data.Type == 'first_serve'),np.where((shot_data.isPointStart == 1) & (swing_data['Result'] == 'In'), 1, 0), np.nan)
shot_data.secondServeIn =np.where((swing_data.Type == 'second_serve') & (shot_data.isPointStart == 1), np.where(swing_data['Result'] == 'In', 1,0), np.nan)

### SwingVision Coord Transformation
court coordinates
swing vision - meters, near side center marks (0,0)
singles court x [-4.1148, 4.1148], y [0, 23.7744]
doubles court x [-5.485, 5.485]

our coordinates - center of net (0,0)
singles court x [-157.5, 157.5], y [-455, 455]

shot_x = (157.5/4.1148) * swing_x
shot_y = (455/11.8872) * swing_y + 455
ratio = 38.2764654418

### firstServeXCoord, firstYServeYCoord, secondServeXCoord, and secondServeyCoord Columns

In [43]:
def first_serve_x_coordinates(stroke, x):
    if stroke == 'first_serve':
        return x * 38.2764654418
    else:
        return np.nan

def first_serve_y_coordinates(stroke, y):
    if stroke == 'first_serve':
        return (y - 11.8872) * 38.2764654418
    else:
        return np.nan
    
shot_data['firstServeXCoord'] = swing_data.apply(lambda row: first_serve_x_coordinates(row['Type'], row['Bounce (x)']), axis=1)
shot_data['firstServeYCoord'] = swing_data.apply(lambda row: first_serve_y_coordinates(row['Type'], row['Bounce (y)']), axis=1)


def second_serve_x_coordinates(stroke, x):
    if stroke == 'second_serve':
        return x * 38.2764654418
    else:
        return np.nan

def second_serve_y_coordinates(stroke, y):
    if stroke == 'second_serve':
        return (y - 11.8872) * 38.2764654418
    else:
        return np.nan
    
shot_data['secondServeXCoord'] = swing_data.apply(lambda row: second_serve_x_coordinates(row['Type'], row['Bounce (x)']), axis=1)
shot_data['secondServeYCoord'] = swing_data.apply(lambda row: second_serve_y_coordinates(row['Type'], row['Bounce (y)']), axis=1)

### firstServeZone and secondServeZone Columns
- serving zones: T, Body, Wide
- Wide: x in [-inf, -105] u [105, inf]
- Body: x in [-105, -52.5] u [52.5, 105]
- T: x in [-52.5, 52.5]

In [44]:
def label_zone(x_coord):
    if x_coord != '':
        x_coord = float(x_coord)
        if (x_coord < -105) | (x_coord > 105):
            return 'Wide'
        elif (-105 <= x_coord <= -52.5) | (52.5 <= x_coord <= 105):
            return 'Body'
        elif -52.5 < x_coord < 52.5:
            return 'T'
    return ''

# convert x coord to serve zone
shot_data.firstServeZone = shot_data.firstServeXCoord.apply(label_zone)
shot_data.secondServeZone = shot_data.secondServeXCoord.apply(label_zone)

In [45]:
shot_data.head()

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes
0,0-0,0-0,0-0,1,0,,,1,,1,Deuce,Player1,Far,1.0,T,14.37932,-198.392394,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rudy Quan,Youcef Rihane,NaN,NaN,NaN,NaN,NaN
1,0-0,0-0,0-0,,1000,,,1,,2,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-0,0-0,0-0,,2000,,,1,,3,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0-0,0-0,0-0,,3000,,,1,,4,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0-0,0-0,0-0,,4000,,,1,,5,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### isAce Column
- WARNING: Not accurate
- FIX: counts double faults as aces

In [46]:
shot_data['isAce'] = np.where((swing_data.Stroke == 'Serve') & 
                              (shot_data.isPointEnd == 1) & 
                              (shot_data.secondServeIn != 0), 1, np.nan)

In [47]:
shot_data[shot_data['isAce'] == 1]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes
486,30-40,2-1,1-0,1,3092000,1,3092000,76,1,1,Ad,Player2,Far,1.0,T,-2.191251,-231.307092,NaN,,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### shotContactX and shotContactY Columns

In [48]:
# Functions to transform the swingvision coordinates
def transf_x_coord_sv_to_shot(sv_col) :
    return sv_col * 38.2764654418
def transf_y_coord_sv_to_shot(sv_col) :
    return (sv_col - 11.8872) * 38.2764654418

# want to convert swingvision coordinates into our own
shot_data['shotContactX'] = transf_x_coord_sv_to_shot(swing_data['Hit (x)'])
shot_data['shotContactY'] = transf_y_coord_sv_to_shot(swing_data['Hit (y)'])

### shotFhBh Column

In [49]:
def classify_shot(stroke):
    if stroke == 'FH Volley':
        return 'Forehand'
    elif stroke == 'BH Volley':
        return 'Backhand'
    elif stroke == 'Forehand':
        return 'Forehand'
    elif stroke == 'Backhand':
        return 'Backhand'
    elif stroke == 'Overhead':
        return 'Forehand'
    else:
        return ''

# Applying the function to the DataFrame
shot_data['shotFhBh'] = swing_data['Stroke'].apply(classify_shot)

### isSlice, isTopspin, isFlat, isKick Columns

In [50]:
shot_data['isSlice'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Slice' else '')
shot_data['isTopspin'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Topspin' else '') # added these metrics
shot_data['isFlat'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Flat' else '') # added these metrics
shot_data['isKick'] = swing_data['Spin'].apply(lambda x: '1' if x == 'Kick' else '') # added these metrics

### isVolley Column
- Volleys are inaccurate

In [51]:
shot_data['isVolley'] = swing_data['Stroke'].apply(lambda x: 1 if x in ['FH Volley', 'BH Volley'] else '') # need to classify shotFhBh when doing isVolley

In [52]:
shot_data[shot_data['isVolley'] == 1]
# 52, 61 75

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
85,30-0,1-1,0-0,,467000,1,467000,14,,5,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,41.818187,-105.700498,NaN,Forehand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
168,40-15,3-2,0-0,,1116000,1,1116000,30,,5,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-60.708541,326.462423,NaN,Forehand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
216,0-0,4-3,0-0,,1436000,1,1436000,37,,6,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-97.921265,331.683754,NaN,Backhand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
240,40-15,4-3,0-0,,1554000,,,41,,5,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,31.284848,-117.005643,NaN,Forehand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
299,15-15,5-4,0-0,,1890000,,,49,,10,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-12.982918,-42.976892,NaN,Forehand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
320,30-40,5-4,0-0,,2001000,,,52,1,7,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-1.237631,-61.203800,NaN,Forehand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
323,30-40,5-4,0-0,,2004000,,,52,1,10,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,21.610548,393.996921,NaN,Backhand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
337,15-15,0-0,1-0,,2189000,1,2189000,55,,5,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-19.854921,-180.051651,NaN,Backhand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
380,0-30,0-1,1-0,,2447000,,,61,,10,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,83.669062,-182.624098,NaN,Forehand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
409,0-0,1-1,1-0,,2608000,,,65,,12,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-106.438698,-270.168039,NaN,Backhand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,


In [53]:
# shot_data[shot_data['pointNumber'].isin([52, 62, 75])]


In [54]:
shot_data[shot_data['pointNumber'] == 37]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
210,0-0,4-3,0-0,1,1417000,,,37,,1,Deuce,Player2,Near,0.0,Wide,-152.449191,-222.856491,NaN,,NaN,NaN,NaN,13.176137,-491.274491,NaN,,1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,
211,0-0,4-3,0-0,1,1426000,,,37,,1,Deuce,Player2,Near,NaN,,NaN,NaN,1.0,T,-5.866021,171.76361,NaN,17.631059,-447.363615,NaN,,1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,
212,0-0,4-3,0-0,,1427000,,,37,,2,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-1.085980,486.118345,NaN,Backhand,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
213,0-0,4-3,0-0,,1428000,,,37,,3,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,90.127182,-480.653193,NaN,Forehand,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
214,0-0,4-3,0-0,,1429000,,,37,,4,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,81.540622,528.783514,NaN,Backhand,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
215,0-0,4-3,0-0,,1430000,,,37,,5,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-54.547178,-512.280386,NaN,Forehand,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
216,0-0,4-3,0-0,,1436000,1,1436000,37,,6,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-97.921265,331.683754,NaN,Backhand,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,


### isOverhead Column
- Overheads are inaccurate

In [55]:
shot_data['isOverhead'] = swing_data['Stroke'].apply(lambda x: 1 if x == 'Overhead' else '')

In [56]:
shot_data[shot_data['isOverhead'] == 1]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
137,15-15,2-2,0-0,,817000,1,817000,23,,5,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,63.130446,225.777444,NaN,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
147,40-15,2-2,0-0,,893000,1,893000,25,,7,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,9.682376,279.528013,NaN,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
238,40-15,4-3,0-0,,1552000,,,41,,3,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,91.642049,-276.517378,NaN,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
322,30-40,5-4,0-0,,2003000,,,52,1,9,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,11.070243,-121.225509,NaN,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
452,30-30,1-1,1-0,,2811000,1,2811000,69,,5,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-44.067809,305.622036,NaN,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
531,40-15,3-1,1-0,,3315000,1,3315000,82,,5,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,21.518531,-181.177018,NaN,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,


### isApproach Column

In [57]:
shot_data[shot_data['pointNumber'] == 2]
# shot_data.query('pointNumber == 2')

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
15,0-15,0-0,0-0,1,44000,,,2,,1,Ad,Player1,Far,1.0,Body,-91.154024,-230.823431,NaN,,NaN,NaN,NaN,11.030703,435.605774,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
16,0-15,0-0,0-0,,45000,,,2,,2,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-165.888555,-531.288823,NaN,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
17,0-15,0-0,0-0,,46000,,,2,,3,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-32.633979,512.494658,NaN,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
18,0-15,0-0,0-0,,47000,,,2,,4,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,147.092514,-529.392454,NaN,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
19,0-15,0-0,0-0,,49000,,,2,,5,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-153.345932,540.084793,NaN,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
20,0-15,0-0,0-0,,50000,1,50000,2,,6,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-81.679604,-418.289157,NaN,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,


In [58]:
# maybe run model to predict (if shot )

point_numbers = shot_data['pointNumber'].unique()

point_numbers





array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97])

### isDropshot Column

In [59]:
# maybe run model to predict

### isExcitingPoint

In [60]:
# maybe run model to predict

### atNetPlayer1 and atNetPlayer2 Columns

In [61]:
# # Define the criteria for being at the player's net
# def is_at_player_net(x, y):
#     return 1 if -245 <= x <= 245 and -157.5 <= y <= 157.5 else ''

# player1Name = shot_data['player1Name'].loc[0] 
# player2Name = shot_data['player2Name'].loc[0]

# # Apply the criteria based on the serverName
# shot_data.loc[shot_data['shotHitBy'] == player1Name, 'atNetPlayer1'] = shot_data.apply(lambda row: is_at_player_net(row['shotContactX'], row['shotContactY']), axis=1)
# shot_data.loc[shot_data['shotHitBy'] == player2Name, 'atNetPlayer2'] = shot_data.apply(lambda row: is_at_player_net(row['shotContactX'], row['shotContactY']), axis=1)


### isLob Column

### shotLocationX and shotLocationY Columns

In [62]:
# Functions to transform the swingvision coordinates
def transf_x_loc(stroke, sv_col):
    if stroke != 'first_serve' and stroke != 'second_serve':
        return sv_col * 38.2764654418
    return np.nan
    
def transf_y_loc(stroke, sv_col):
    if stroke != 'first_serve' and stroke != 'second_serve':
        return (sv_col - 11.8872) * 38.2764654418
    return np.nan

# want to convert swingvision coordinates into our own
shot_data['shotLocationX'] = swing_data.apply(lambda x: transf_x_loc(x['Type'], x['Bounce (x)']), axis=1)
shot_data['shotLocationY'] = swing_data.apply(lambda x: transf_y_loc(x['Type'], x['Bounce (y)']), axis=1)

### shotDirection column

In [63]:
# down the line --> switches btwn deuce and ad
# crosscourt --> remains on same side
shot_data['shotDirection'] = np.where((shot_data.shotContactX * shot_data.shotLocationX > 0) & (shot_data.shotInRally != 1), 
"Down the Line", 
    np.where((shot_data.shotInRally != 1), 'Crosscourt', ''))

### isWinner Column

In [64]:
shot_data.isWinner = np.where((shot_data.isPointEnd == 1) & (shot_data.secondServeIn != '0') &
                              (swing_data.Result == 'In'), 1, np.nan)


In [65]:
shot_data[shot_data['isWinner'] == 1]
# shot_data[330:345]
# shot_data[shot_data['pointNumber'] == 10]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
20,0-15,0-0,0-0,,50000,1,50000,2,,6,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-81.679604,-418.289157,Down the Line,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-146.381261,289.702051,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
33,15-40,0-0,0-0,,130000,1,130000,5,1,4,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-103.368274,-549.365688,Crosscourt,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,91.593936,445.558459,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
85,30-0,1-1,0-0,,467000,1,467000,14,,5,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,41.818187,-105.700498,Crosscourt,Forehand,,1,,NaN,NaN,NaN,NaN,NaN,NaN,-143.331966,315.003751,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
102,30-0,2-1,0-0,,646000,1,646000,18,,6,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-191.717285,572.026465,Crosscourt,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,99.876274,-219.623699,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
130,0-0,2-2,0-0,,753000,1,753000,21,,22,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,138.684935,-512.267104,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,154.595390,418.471850,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
137,15-15,2-2,0-0,,817000,1,817000,23,,5,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,63.130446,225.777444,Down the Line,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,102.807983,-82.801755,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
147,40-15,2-2,0-0,,893000,1,893000,25,,7,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,9.682376,279.528013,Crosscourt,Forehand,,,1,NaN,NaN,NaN,NaN,NaN,NaN,-131.146500,-212.312128,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
152,15-0,3-2,0-0,,1015000,1,1015000,27,,3,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,98.622337,525.245620,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,140.081873,-419.093805,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
168,40-15,3-2,0-0,,1116000,1,1116000,30,,5,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-60.708541,326.462423,Crosscourt,Forehand,,1,,NaN,NaN,NaN,NaN,NaN,NaN,115.810537,-337.796391,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
173,0-0,3-3,0-0,,1158000,1,1158000,31,,4,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,63.283590,220.639326,Crosscourt,Backhand,1,,,NaN,NaN,NaN,NaN,NaN,NaN,-0.975514,-294.596730,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,


### isErrorWideR Column

In [66]:
def wide_right_function(side, x, y, end):
    if (side == 'far' and x < -157.5 and end == 1) or (side == 'near' and x > 157.5 and end == 1):
        return 1
    return np.nan

# Assign 'isErrorWideR' using values from both 'swing_data' and 'shot_data'
shot_data['isErrorWideR'] = shot_data.apply(lambda x: wide_right_function(swing_data.loc[x.name, 'Hit Side'], 
                                                                x['shotLocationX'], x['shotLocationY'], x['isPointEnd']), axis=1)

shot_data[shot_data['isErrorWideR'] == 1]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
37,0-0,0-1,0-0,,187000,1,187000,6,,4,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,109.404396,-432.789047,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,177.930293,395.562314,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
149,0-0,3-2,0-0,,1000000,1,1000000,26,,2,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,34.015070,-449.779817,Down the Line,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,202.895505,183.638271,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
209,40-30,3-3,0-0,,1334000,1,1334000,36,,2,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,17.169712,546.816896,Crosscourt,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-217.585400,-444.346894,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
241,40-15,4-3,0-0,,1556000,1,1556000,41,,6,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,175.700383,477.160083,Crosscourt,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-205.295554,-432.576039,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
369,0-15,0-1,1-0,,2398000,1,2398000,60,,9,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,68.317902,-466.633022,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,162.358929,275.370729,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,


In [67]:
def wide_right_function(side, x, y, end):
    if (side == 'far' and x < -157.5 and y > -405 and y < 0 and end == 1) or (side == 'near' and x > 157.5 and y < 405 and y > 0 and end == 1):
        return 1
    return np.nan

# Assign 'isErrorWideR' using values from both 'swing_data' and 'shot_data'
shot_data['isErrorWideR'] = shot_data.apply(lambda x: wide_right_function(swing_data.loc[x.name, 'Hit Side'], 
                                                                x['shotLocationX'], x['shotLocationY'], x['isPointEnd']), axis=1)

In [68]:
shot_data[shot_data['isErrorWideR'] == 1]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
37,0-0,0-1,0-0,,187000,1,187000,6,,4,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,109.404396,-432.789047,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,177.930293,395.562314,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
149,0-0,3-2,0-0,,1000000,1,1000000,26,,2,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,34.015070,-449.779817,Down the Line,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,202.895505,183.638271,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
369,0-15,0-1,1-0,,2398000,1,2398000,60,,9,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,68.317902,-466.633022,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,162.358929,275.370729,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,


### isErrorWideL Column

In [69]:
def wide_left_function(side, x, y, end):
    if (side == 'far' and x > 157.5 and y > -405 and y < 0 and end == 1) or (side == 'near' and x < -157.5 and y < 405 and y > 0 and end == 1):
        return 1
    return np.nan

# Assign 'isErrorWideR' using values from both 'swing_data' and 'shot_data'
shot_data['isErrorWideL'] = shot_data.apply(lambda x: wide_left_function(swing_data.loc[x.name, 'Hit Side'], 
                                                                x['shotLocationX'], x['shotLocationY'], x['isPointEnd']), axis=1)

In [70]:
shot_data.query('isErrorWideL == 1')

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
88,40-0,1-1,0-0,,503000,1,503000,15,,2,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,150.525492,505.286931,Down the Line,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,181.075662,-364.734640,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
342,15-30,0-0,1-0,,2212000,1,2212000,56,,5,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,167.832502,-520.683716,Crosscourt,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-169.131376,392.065682,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
517,15-15,3-1,1-0,,3231000,1,3231000,80,,6,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-39.007125,526.258683,Crosscourt,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,169.113041,-354.888402,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,


### isErrorNet Column

In [71]:
shot_data.isErrorNet = np.where((swing_data.Result == 'Net'), 1, np.nan)

### isErrorLong Column

In [72]:
shot_data['isErrorLong'] = np.where((swing_data['Result'] == 'Out') & (shot_data['shotLocationY'].abs() > 455), 1, np.nan)

In [73]:
shot_data[shot_data['isErrorLong'] == 1]

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
3,0-0,0-0,0-0,,3000,,,1,,4,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-73.878554,-516.058120,Crosscourt,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,107.496582,480.407037,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
59,15-30,0-1,0-0,,298000,1,298000,9,,7,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,57.685887,514.961767,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,135.932437,-511.711445,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
94,0-0,2-1,0-0,,601000,1,601000,16,,6,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,166.390015,526.593755,Crosscourt,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-79.044116,-488.613933,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
108,40-15,2-1,0-0,,693000,1,693000,20,,4,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,122.223912,511.705244,Down the Line,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,113.376537,-516.193274,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
140,30-15,2-2,0-0,,864000,1,864000,24,,2,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-146.471632,-486.725448,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-106.390469,455.887708,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
171,0-0,3-3,0-0,,1155000,,,31,,2,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-89.448272,492.106507,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-79.583470,-496.869477,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
227,0-15,4-3,0-0,,1472000,1,1472000,38,,10,Deuce,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-204.747512,565.763019,Down the Line,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,-66.299852,-478.467760,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
300,15-15,5-4,0-0,,1892000,1,1892000,49,,11,Deuce,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-125.516109,490.621916,Crosscourt,Forehand,,,,NaN,NaN,NaN,NaN,NaN,NaN,10.122671,-491.770287,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1,
326,0-0,0-0,1-0,,2131000,,,53,,2,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-98.311035,553.200569,Crosscourt,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,77.875191,-475.553505,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,
332,15-0,0-0,1-0,,2161000,1,2161000,54,,3,Ad,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-56.489939,-505.969707,Crosscourt,Backhand,,,,NaN,NaN,NaN,NaN,NaN,NaN,66.115972,508.054932,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,,


### Group First Serve and Second Serve Columns

In [74]:
# All columns
default_cols = ['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes', 'isTopspin', 'isFlat', 'isKick']

# Assign all columns to have value be taken from the first serve row
agg_dict = {col: 'first' for col in default_cols}

# Reassign select columns to have value be taken form the second serve row
agg_dict.update({'isPointEnd': 'last', 
                 'pointEndTime': 'last', 
                 'secondServeIn' : 'last',
                 'secondServeZone' : 'last',
                 'secondServeXCoord' : 'last', 
                 'secondServeYCoord' : 'last',
                 'isAce' : 'last', 
                 'shotContactX' : 'last', 
                 'shotContactY' : 'last',
                 'isWinner' : 'last',
                 'isErrorWideR' : 'last',
                 'isErrorWideL' : 'last', 
                 'isErrorNet' : 'last',
                 'isErrorLong' : 'last'
                })

# Group by isPointStart and pointNumber
grouped_df = shot_data.groupby(['shotInRally', 'pointNumber'], as_index=False).agg(agg_dict)
shot_data = grouped_df.sort_values(by=['pointNumber', 'shotInRally'], ascending=[True, True]).reset_index(drop = True)

### Save as CSV

In [75]:
player1NameNoSpace = str(shot_data.iloc[0]['player1Name']).replace(" ", "")
player2NameNoSpace = str(shot_data.iloc[0]['player2Name']).replace(" ", "")

shot_data.to_csv(f'swingvision_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)

### Error Analysis

In [76]:
point_error = shot_data[(shot_data['isPointEnd'] != 1) & (shot_data['isPointStart'] != 1) &
          ((shot_data['isWinner'] == 1) | 
          (shot_data['isErrorNet'] == 1) | 
          (shot_data['isErrorLong'] == 1) |
          (shot_data['isErrorWideL'] == 1) |
          (shot_data['isErrorWideR'] == 1))]

point_error_numbers = point_error['pointNumber'].to_list()

if len(point_error) > 0:
    display(point_error)
    raise ValueError('Manually check points', point_error_numbers)

,pointScore,gameScore,setScore,isPointStart,pointStartTime,isPointEnd,pointEndTime,pointNumber,isBreakPoint,shotInRally,side,serverName,serverFarNear,firstServeIn,firstServeZone,firstServeXCoord,firstServeYCoord,secondServeIn,secondServeZone,secondServeXCoord,secondServeYCoord,isAce,shotContactX,shotContactY,shotDirection,shotFhBh,isSlice,isVolley,isOverhead,isApproach,isDropshot,isExcitingPoint,atNetPlayer1,atNetPlayer2,isLob,shotLocationX,shotLocationY,isWinner,isErrorWideR,isErrorWideL,isErrorNet,isErrorLong,clientTeam,Date,Division,Event,lineupPosition,matchDetails,matchVenue,opponentTeam,player1Name,player2Name,player1Hand,player2Hand,Round,Surface,Notes,isTopspin,isFlat,isKick
3,0-0,0-0,0-0,,3000,,,1,,4,Ad,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-73.878554,-516.058120,Crosscourt,Backhand,,,,None,None,None,None,None,None,107.496582,480.407037,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
163,0-0,3-3,0-0,,1155000,,,31,,2,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-89.448272,492.106507,Down the Line,Forehand,,,,None,None,None,None,None,None,-79.583470,-496.869477,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
206,0-0,4-3,0-0,,1430000,,,37,,5,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-54.547178,-512.280386,Down the Line,Forehand,,,,None,None,None,None,None,None,-90.651990,-13.957590,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
314,0-0,0-0,1-0,,2131000,,,53,,2,Deuce,Player1,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-98.311035,553.200569,Crosscourt,Backhand,,,,None,None,None,None,None,None,77.875191,-475.553505,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
362,0-30,0-1,1-0,,2446000,,,61,,8,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,172.686494,523.201580,Down the Line,Backhand,1,,,None,None,None,None,None,None,172.686494,476.933448,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,
368,0-40,0-1,1-0,,2482000,,,62,1,3,Ad,Player2,Near,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-25.657021,-486.306168,Down the Line,Backhand,,,,None,None,None,None,None,None,-93.471703,457.004653,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,
421,15-15,1-1,1-0,,2732000,,,67,,3,Deuce,Player1,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-144.467897,436.091196,Down the Line,Forehand,,,,None,None,None,None,None,None,-21.282825,-510.493755,NaN,NaN,NaN,NaN,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,,
463,30-30,2-1,1-0,,3066000,,,75,,6,Ad,Player2,Far,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,-113.517049,-315.436581,Down the Line,Backhand,,1,,None,None,None,None,None,None,-98.606491,-358.001389,NaN,NaN,NaN,1.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,1,


ValueError: ('Manually check points', [1, 31, 37, 53, 61, 62, 67, 75])

In [ ]:
shot_data[shot_data['pointNumber'] == 75]

In [ ]:
# shot_data[(shot_data['isPointEnd'] != 1) & (shot_data['isErrorLong'] == 1)]

In [ ]:
shot_data[shot_data['pointNumber'] == ]

### Points

In [ ]:
# # ad scoring?

# ad_scoring = False

In [ ]:
# # want to record the score every time a point ends
# # points: server - returner
# # games: ucla (player1) - opp
# # sets: ucla (player1) - opp
# points = np.zeros(2)
# games = np.zeros(2)
# sets = np.zeros(2)
# pt_values = [0, 15, 30, 40]



# shot_data.loc[0,"pointScore"] = f"{pt_values[int(points[0])]} - {pt_values[int(points[1])]}"
# shot_data.loc[0,"gameScore"] = f"{games[0]} - {games[1]}"
# shot_data.loc[0,"setScore"] = f"{sets[0]} - {sets[1]}"

# shot_data["isBreakPoint"] = ''

# error_cols = [x for x in shot_data.columns if "isError" in x]

# for i in range(0, len(shot_data.pointScore) - 1):
#     if shot_data.loc[i+1, "isPointStart"] == 1: # means we gotta update pts
#         # determine point score by checking last shot
#         if shot_data.loc[i, "isWinner"] == "1":
#             # check if player 1 or 2 won pt
#             pt_winner_player_num = (np.where(shot_data.loc[i, "Player"] == shot_data.loc[0,"player1Name"], '1', '2'))
#         elif shot_data.loc[i, 'secondServeIn'] == "0": # double fault
#             pt_winner_player_num = (np.where(shot_data.loc[i, "Player"] == shot_data.loc[0,"player1Name"], '2', '1'))
#         elif any(shot_data.loc[i,error_cols] == "1"):
#             # winner is the player who did NOT hit that shot
#             pt_winner_player_num = (np.where(shot_data.loc[i, "Player"] == shot_data.loc[0,"player1Name"], '2', '1'))
#         else:
#             print("no pt recorded at row ", i)

#         if shot_data.loc[i, "serverName"] is not None:
#             didServerWinPt = shot_data.loc[i, "serverName"][-1] == pt_winner_player_num
#         else:
#             print(f"Server name is None at row {i}. Skipping this point.")
#             continue  # Skip this point if server name is None
        
#         if didServerWinPt:
#             points[0] += 1
#         else:
#             points[1] += 1


#         if ad_scoring: # checks if need to win by 2
#             if any(points > 3) and abs(points[0] - points[1]) >= 2:
#                 game_winner = np.argmax(points)  # Find who won the game
#                 games[game_winner] += 1
#                 points = np.zeros(2)  # Reset point values
#             if any(games > 5) and abs(games[0] - games[1]) >= 2:
#                 set_winner = np.argmax(games)  # Find who won the set
#                 sets[set_winner] += 1
#                 games = np.zeros(2)  # Reset game values   
#         else:
#             if points[1] == 3: # if the returner has 40 pts and can win the game
#                 shot_data.loc[i + 1, 'isBreakPoint'] = '1'
#             if any(points > 3):
#                 game_winner = np.argmax(points)  # Find who won the game
#                 games[game_winner] += 1
#                 points = np.zeros(2)  # Reset point values
#             if any(games > 5):
#                 set_winner = np.argmax(games)  # Find who won the set
#                 sets[set_winner] += 1
#                 games = np.zeros(2)  # Reset game values   

#     # Update the scores in the shot_data DataFrame
#     shot_data.loc[i+1,"pointScore"] = f"{pt_values[int(points[0])]} - {pt_values[int(points[1])]}"
#     shot_data.loc[i+1,"gameScore"] = f"{int(games[0])} - {int(games[1])}"
#     shot_data.loc[i+1,"setScore"] = f"{int(sets[0])} - {int(sets[1])}"


# # Additional comments for further updates:
# # - Tiebreak scenarios are not yet handled and need to be accounted for in future versions.